# Динамика по месяцам
Пример расчета динамики показателей по месяцам по категории в детализации по рекламодателю/бренду/продукту

## Описание задачи и условий расчета
- Период: 01.07.2022-31.12.2022 по месяцам
- ЦА: Все 4+
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index
- Ролики: ролик статус - Реальный
- Статистики: StandRtg% (20) (среднее)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# Обратимся к словарю статусов события
cats.get_tv_issue_status()

# Для "Реальный" будем использовать id R

In [ ]:
# В словаре товарных категорий найдем id категории КОЛБАСЫ И ВЕТЧИНЫ
cats.get_tv_article(name=['КОЛБАСЫ И ВЕТЧИНЫ'])

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2022-07-01', '2022-12-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = 'tvNetId IN (1,2,4,10,11,12,13,16,40,60,83,84,86,204,205,206,255,257,258,259,260,286,326,329,340,356,376,393,502,545,348)'

# Указываем фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов: статус - реальный
ad_filter = 'adIssueStatusId = R and articleLevel4Id = 4973 and adTypeId IN (1,10,23,25,5,15,24)'

# Указываем список срезов (группировок)
slices = [
    'articleLevel4Name', #товарная категория 4
    'advertiserName', #рекламодатель
    'brandName', #бренд
    'modelName', #продукт
    'researchMonth' #месяц
]

# Указываем список статистик для расчета
statistics = ['StandRtgPerSum']

# Задаем условия сортировки: Телекомпания (от а до я), рекламодатель (от а до я), Бренд (от а до я)
sortings = {'researchMonth':'ASC','articleLevel4Name':'ASC','advertiserName':'ASC','brandName':'ASC','modelName':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "issueType": "AD", #Тип события - Ролики
    "standRtg" : {
      "useRealDuration" : True, #расчет по реальной длительности ролика.
      "standardDuration" : 20 #стандартная длительность 20 сек.
    }
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_crosstab = mtask.wait_task(mtask.send_crosstab_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_crosstab))

df = df[slices+statistics]
df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - срезы, столбцы - месяцы, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['articleLevel4Name','advertiserName','brandName','modelName'], 
                        columns=['researchMonth'])
df

In [ ]:
# Опционально: поменять местами уровни в столбцах
df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('08_monthly_dynamics')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)